In [ ]:
pip install python-chess==0.31.

In [ ]:
import chess
import chess.engine
import random
import numpy
import matplotlib.pyplot as plt

In [ ]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

## **Extracting evaluation**

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/preprocessed15.csv", header=None)

In [ ]:
y = dataset[64].values
y.shape

**Normalization**

In [ ]:
normalizedy = (y-min(y))/(max(y)-min(y))
normalizedy

In [ ]:
plt.hist(y, bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

In [ ]:
plt.hist(normalizedy, bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

# **Importing training data**

In [ ]:
traindata = numpy.load('/content/trainingdata15.npy')
print(traindata.shape)

# **Building Model**

In [ ]:
def build_model(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))
    x = board3d
    for _ in range(conv_depth):
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_last')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(16, 'relu')(x)
    x = layers.Dense(1, 'sigmoid')(x)

    return models.Model(inputs=board3d, outputs=x)

In [ ]:
model = build_model(16, 1)

In [ ]:
import tensorflow.keras.callbacks as callbacks

In [ ]:
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()
model.fit(traindata,normalizedy,
          batch_size=1024,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

In [ ]:
model.save('model19.h5')

 **3d Matrix**

In [ ]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1


  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [ ]:
!pip install keras
from keras.models import load_model



In [ ]:
model = load_model('model18.h5')

In [ ]:
def eval(board):
    board3d = split_dims(board)
    board3d = numpy.expand_dims(board3d, 0)
    return model.predict(board3d)[0][0]

# **Monte Carlo Tree Search**

In [ ]:
class Node:
    
    def __init__(self, state, move=None, parent=None):
        self.move = move
        self.state = state
        self.parent = parent
        self.unexplored_moves = list(self.state.legal_moves)
        self.children = []
        self.visits = 0
        self.wins = 0
        
    def add_child(self, state, move):
        child_node = Node(state, move, self)
        self.children.append(child_node)
        self.unexplored_moves.remove(move)
        return child_node
        
    def UCT_select_child(self):
        s = sorted(self.children, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def Update(self, result):
        self.visits += 1
        self.wins += result

In [ ]:
import random
from math import sqrt, log
def UCT(rootstate, itermax, depthmax):
    rootnode = Node(state = rootstate)
    for i in range(itermax):
        node = rootnode
        depth = 0
        state = rootstate.copy()

        # Select
        while node.unexplored_moves == [] and node.children != []: # node is fully expanded and non-terminal
            node = node.UCT_select_child()
            state.push(node.move)

        # Expand
        if node.unexplored_moves != []:
            m = random.choice(node.unexplored_moves) 
            state.push(m)
            node = node.add_child(state,m) 
            depth += 1

        # Rollout 
       # while list(state.legal_moves) != [] and depth < depthmax: 
        #    state.push(random.choice(list(state.legal_moves)))
         #   depth += 1

        # Backpropagate
        while node != None: 
            result = eval(state)
            
            node.Update(result) 
            node = node.parent

    return sorted(rootnode.children, key = lambda c: c.visits)[-1].move

In [ ]:
import time
from IPython.display import clear_output, display
import chess.pgn

board = chess.Board()

def play_game(player1, player2):
    while not board.is_game_over():
        if board.turn == chess.WHITE:
            player1(board)
        else:
            player2(board)
        clear_output(wait=True)
        display(board)
        time.sleep(0.1)
    game = chess.pgn.Game.from_board(board)
    print(game)

In [ ]:
import random

board = chess.Board()

def mcts_player(board):
    move = None
    for move_choice in board.legal_moves:
        copy = board.copy()
        copy.push(move_choice)
        if copy.is_game_over():
            move = move_choice
            board.push(move)
            return
    if random.random() < 0.0001:
        move = random.choice(list(board.legal_moves))
    else:
        move = UCT(board, 10, 10)
    board.push(move)

In [ ]:
def human_player(board):
    while True:
        move = input("Input Your Move:")
        if move == "q":
            raise KeyboardInterrupt
        try:
            board.push_san(move)
            break 
        except Exception as e:
            print(e)

In [ ]:
play_game(mcts_player, mcts_player)